# Run pyCIAM

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from shared import (
    AUTHOR,
    CONTACT,
    DIR_SCRATCH,
    HISTORY,
    PATH_OUTPUTS,
    PATH_PARAMS,
    PATH_REFA,
    PATH_SLIIDERS,
    PATH_SLIIDERS_SEG,
    PATH_SLR_INT,
    PATHS_SURGE_LOOKUP,
    start_dask_cluster,
)

from pyCIAM.run import execute_pyciam

/home/jovyan/git-repos/ice-sheet-impacts/code/shared.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd
/srv/conda/envs/notebook/lib/python3.10/site-packages/google/cloud/storage/transfer_manager.py:30: UserWarning: The module `transfer_manager` is a preview feature. Functionality and API may change. This warning will be r

In [3]:
TMPPATH = DIR_SCRATCH / "pyCIAM_results_quantiles_prechunked.zarr"

# When running on larger/scalable dask cluster, may wish to specify number of workers
# Default is LocalCluster which will use the number of CPUs available on local machine
# N_WORKERS = None
N_WORKERS = 400

SEG_CHUNKSIZE = 3

SEG_ADM_SUBSET = None

DESCRIPTION = "Projected coastal damages from pyCIAM, using quantiles of SLR scenarios."

In [4]:
client, cluster = start_dask_cluster()
cluster.scale(N_WORKERS)
cluster

In [ ]:
finished = execute_pyciam(
    PATH_PARAMS,
    PATH_SLIIDERS,
    [PATH_SLR_INT],
    ["is"],
    PATH_REFA,
    econ_input_path_seg=PATH_SLIIDERS_SEG,
    output_path=PATH_OUTPUTS,
    seg_var_subset=SEG_ADM_SUBSET,
    surge_input_paths=PATHS_SURGE_LOOKUP,
    tmp_output_path=TMPPATH,
    quantiles=[0.5],
    dask_client_func=lambda: client,
    extra_attrs={
        "author": AUTHOR,
        "contact": CONTACT,
        "description": DESCRIPTION,
        "history": HISTORY,
    },
)